## Pydantic Nested Models


#### 1. Model Nesting


In [1]:
from pydantic import BaseModel


# 1. Define the Inner Model (Address)
class Address(BaseModel):
    street: str
    city: str
    zip_code: str


# 2. Define the Outer Model (User)
class User(BaseModel):
    user_id: int
    name: str

    # Nesting occurs here: 'address' field is an instance of the Address model
    address: Address

In [2]:
valid_data = {
    "user_id": 101,
    "name": "Yash Jain",
    "address": {
        "street": "123 Main St",
        "city": "Springfield",
        "zip_code": "62704",
    },
}

# Pydantic validates the whole structure
user1 = User(**valid_data)

In [3]:
print(f"User Name: {user1.name}")
# Access nested fields using dot notation
print(f"User City: {user1.address.city}")

# Verify the type of the nested field
print(f"Address Type: {type(user1.address)}")

User Name: Yash Jain
User City: Springfield
Address Type: <class '__main__.Address'>


---

#### 2. List of Nested Models

-   Entity type as a List of Pydantic Model


In [4]:
from pydantic import BaseModel
from typing import List


# 1. Define the Inner Model (Product)
class Product(BaseModel):
    product_name: str
    price: float
    quantity: int


# 2. Define the Outer Model (Order)
class Order(BaseModel):
    order_id: str
    customer_name: str

    # Nesting a List of Models: 'items' is a list where each element MUST be a Product
    items: List[Product]

In [5]:
order_data = {
    "order_id": "ORD-001",
    "customer_name": "Bob Green",
    "items": [
        {
            "product_name": "Laptop Charger",
            "price": 45.00,
            "quantity": 1,
        },
        {
            "product_name": "Mousepad",
            "price": 12.50,
            "quantity": 2,
        },
    ],
}

order1 = Order(**order_data)

In [6]:
print(f"Order ID: {order1.order_id}")
print(f"Total Items in Order: {len(order1.items)}")

# Access the first nested item and its attributes
first_item = order1.items[0]
print(f"First Item Name: {first_item.product_name}")

Order ID: ORD-001
Total Items in Order: 2
First Item Name: Laptop Charger


In [7]:
from pydantic import ValidationError

# Pydantic validation handles errors at any level
try:
    Order(
        order_id="ORD-002",
        customer_name="Invalid Order",
        items=[
            {
                "product_name": "Valid Item",
                "price": 10.0,
                "quantity": 1,
            },
            {
                "product_name": "Invalid Item",
                "price": "ten dollars",
                "quantity": 1,
            },  # Invalid price type
        ],
    )
except ValidationError as e:
    print("\n--- Validation Error in Nested List ---")
    # The error location clearly shows the failure occurred in 'items', at index 1, in field 'price'
    print(e.errors()[0]["loc"])
    print(e.errors()[0]["type"])
    print(e.errors()[0]["msg"])


--- Validation Error in Nested List ---
('items', 1, 'price')
float_parsing
Input should be a valid number, unable to parse string as a number


---

#### 3. Deep and Optional Nesting


In [8]:
from pydantic import BaseModel
from typing import Optional


# 1. Innermost Model (GeoLocation)
class GeoLocation(BaseModel):
    latitude: float
    longitude: float


# 2. Intermediate Model (RestaurantDetails)
class RestaurantDetails(BaseModel):
    rating: float

    # Deep Nesting: Nesting GeoLocation here
    location: GeoLocation


# 3. Outer Model (Venue)
class Venue(BaseModel):
    name: str

    # Optional Nesting: The details block might not always be present
    details: Optional[RestaurantDetails] = None

In [9]:
venue_data_full = {
    "name": "The Great Bistro",
    "details": {
        "rating": 4.5,
        "location": {
            "latitude": 34.0522,
            "longitude": -118.2437,
        },
    },
}
venue1 = Venue(**venue_data_full)
print(f"Venue: {venue1.name}")
print(f"Rating: {venue1.details.rating}")
# Accessing deeply nested attribute
print(f"Latitude: {venue1.details.location.latitude}")

Venue: The Great Bistro
Rating: 4.5
Latitude: 34.0522


In [10]:
# Input data omits the 'details' field
venue_data_partial = {
    "name": "Pop-Up Cafe",
}
venue2 = Venue(**venue_data_partial)
print(f"\nVenue: {venue2.name}")
# The 'details' field is None
print(f"Details status: {venue2.details}")


Venue: Pop-Up Cafe
Details status: None


---

#### 4. Self-Referencing (Recursive) Models


In [11]:
from pydantic import BaseModel
from typing import List


# 1. Define the Recursive Model
class TreeNode(BaseModel):
    id: int
    name: str

    # The 'children' field is a list of other TreeNode instances
    children: List["TreeNode"] = None

In [12]:
# Input data defining a simple tree structure:
# Root -> Child A -> Grandchild X
#      -> Child B
tree_data = {
    "id": 1,
    "name": "Root",
    "children": [
        {
            "id": 2,
            "name": "Child A",
            "children": [
                {
                    "id": 4,
                    "name": "Grandchild X",
                },
            ],
        },
        {
            "id": 3,
            "name": "Child B",
            # 'children' defaults to [] if not provided
        },
    ],
}

root_node = TreeNode(**tree_data)

print(f"Root Node: {root_node.name}")
print(f"Number of direct children: {len(root_node.children)}")
print(f"Name of Grandchild: {root_node.children[0].children[0].name}")

Root Node: Root
Number of direct children: 2
Name of Grandchild: Grandchild X
